In [ ]:
# 0800-1223425

In [24]:
## instalar librerias para tvm
!pip install apache-tvm --pre
!pip install typing_extensions
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tlcpack (from versions: none)
ERROR: No matching distribution found for tlcpack


In [25]:
import tvm

In [26]:
!wget https://github.com/onnx/models/raw/b9a54e89508f101a1611cd64f4ef56b9cb62c7cf/vision/classification/resnet/model/resnet50-v2-7.onnx

--2023-05-22 18:41:28--  https://github.com/onnx/models/raw/b9a54e89508f101a1611cd64f4ef56b9cb62c7cf/vision/classification/resnet/model/resnet50-v2-7.onnx
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/onnx/models/b9a54e89508f101a1611cd64f4ef56b9cb62c7cf/vision/classification/resnet/model/resnet50-v2-7.onnx [following]
--2023-05-22 18:41:29--  https://media.githubusercontent.com/media/onnx/models/b9a54e89508f101a1611cd64f4ef56b9cb62c7cf/vision/classification/resnet/model/resnet50-v2-7.onnx
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102442450 (98M) [application/octet-strea

In [29]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 67.1 MB/s eta 0:00:00


In [30]:
# This may take several minutes depending on your machine
!tvmc compile --target "llvm" --input-shapes "data:[1,3,224,224]" --output resnet50-v2-7-tvm.tar resnet50-v2-7.onnx

In [31]:
!mkdir model
!tar -xvf resnet50-v2-7-tvm.tar -C model
!ls model

mod.so
mod.json
mod.params
mod.json  mod.params  mod.so


In [32]:
from tvm.contrib.download import download_testdata
from PIL import Image
import numpy as np

img_url = "https://s3.amazonaws.com/model-server/inputs/kitten.jpg"
img_path = download_testdata(img_url, "imagenet_cat.png", module="data")

# Resize it to 224x224
resized_image = Image.open(img_path).resize((224, 224))
img_data = np.asarray(resized_image).astype("float32")

# ONNX expects NCHW input, so convert the array
img_data = np.transpose(img_data, (2, 0, 1))

# Normalize according to ImageNet
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_stddev = np.array([0.229, 0.224, 0.225])
norm_img_data = np.zeros(img_data.shape).astype("float32")
for i in range(img_data.shape[0]):
      norm_img_data[i, :, :] = (img_data[i, :, :] / 255 - imagenet_mean[i]) / imagenet_stddev[i]

# Add batch dimension
img_data = np.expand_dims(norm_img_data, axis=0)

# Save to .npz (outputs imagenet_cat.npz)
np.savez("imagenet_cat", data=img_data)

In [34]:
!tvmc run --inputs imagenet_cat.npz --output predictions.npz resnet50-v2-7-tvm.tar

2023-05-22 18:48:47.868 INFO load_module /tmp/tmp5kglqhux/mod.so


In [35]:
import os.path
import numpy as np

from scipy.special import softmax

from tvm.contrib.download import download_testdata

# Download a list of labels
labels_url = "https://s3.amazonaws.com/onnx-model-zoo/synset.txt"
labels_path = download_testdata(labels_url, "synset.txt", module="data")

with open(labels_path, "r") as f:
    labels = [l.rstrip() for l in f]

output_file = "predictions.npz"

# Open the output and read the output tensor
if os.path.exists(output_file):
    with np.load(output_file) as data:
        scores = softmax(data["output_0"])
        scores = np.squeeze(scores)
        ranks = np.argsort(scores)[::-1]

        for rank in ranks[0:5]:
            print("class='%s' with probability=%f" % (labels[rank], scores[rank]))

class='n02123045 tabby, tabby cat' with probability=0.610552
class='n02123159 tiger cat' with probability=0.367180
class='n02124075 Egyptian cat' with probability=0.019365
class='n02129604 tiger, Panthera tigris' with probability=0.001273
class='n04040759 radiator' with probability=0.000261


In [36]:
# The default search algorithm requires xgboost, see below for further
# details on tuning search algorithms
!pip install xgboost
!tvmc tune --target "llvm" --output resnet50-v2-7-autotuner_records.json resnet50-v2-7.onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[Task  1/25]  Current/Best:   10.20/  20.32 GFLOPS | Progress: (40/40) | 58.73 s Done.
[Task  2/25]  Current/Best:    2.99/  16.06 GFLOPS | Progress: (40/40) | 37.26 s Done.
[Task  3/25]  Current/Best:    8.80/  18.64 GFLOPS | Progress: (40/40) | 45.76 s Done.
[Task  4/25]  Current/Best:    5.29/  17.42 GFLOPS | Progress: (40/40) | 52.50 s Done.
[Task  5/25]  Current/Best:    2.33/  19.91 GFLOPS | Progress: (40/40) | 44.90 s Done.
[Task  6/25]  Current/Best:    2.71/  15.15 GFLOPS | Progress: (40/40) | 56.07 s Done.
[Task  7/25]  Current/Best:   13.25/  18.24 GFLOPS | Progress: (40/40) | 43.57 s Done.
[Task 10/25]  Current/Best:   17.99/  19.34 GFLOPS | Progress: (40/40) | 44.87 s Done.
[Task 11/25]  Current/Best:    7.56/  20.36 GFLOPS | Progress: (40/40) | 46.88 s Done.
[Task 12/25]  Current/Best:    9.14/  19.65 GFLOPS | Progress: (40/40) | 57.94 s Done.
[Task 13/25]  Current/Best:   1

In [38]:
!tvmc compile --target "llvm" --tuning-records resnet50-v2-7-autotuner_records.json --output resnet50-v2-7-tvm_autotuned.tar resnet50-v2-7.onnx

In [39]:
!tvmc run --inputs imagenet_cat.npz --output predictions.npz resnet50-v2-7-tvm_autotuned.tar

2023-05-22 19:24:27.401 INFO load_module /tmp/tmprkgccobp/mod.so


In [40]:
labels_url = "https://s3.amazonaws.com/onnx-model-zoo/synset.txt"
labels_path = download_testdata(labels_url, "synset.txt", module="data")

with open(labels_path, "r") as f:
    labels = [l.rstrip() for l in f]

output_file = "predictions.npz"

# Open the output and read the output tensor
if os.path.exists(output_file):
    with np.load(output_file) as data:
        scores = softmax(data["output_0"])
        scores = np.squeeze(scores)
        ranks = np.argsort(scores)[::-1]

        for rank in ranks[0:5]:
            print("class='%s' with probability=%f" % (labels[rank], scores[rank]))

class='n02123045 tabby, tabby cat' with probability=0.610552
class='n02123159 tiger cat' with probability=0.367179
class='n02124075 Egyptian cat' with probability=0.019365
class='n02129604 tiger, Panthera tigris' with probability=0.001273
class='n04040759 radiator' with probability=0.000261


In [41]:
!tvmc run \
--inputs imagenet_cat.npz \
--output predictions.npz  \
--print-time \
--repeat 100 \
resnet50-v2-7-tvm_autotuned.tar

2023-05-22 19:27:17.158 INFO load_module /tmp/tmpbivzlo3f/mod.so
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  537.0951     457.1980     903.6262     443.9307     143.7449  
               


In [42]:
!tvmc run \
--inputs imagenet_cat.npz \
--output predictions.npz  \
--print-time \
--repeat 100 \
resnet50-v2-7-tvm.tar

2023-05-22 19:28:15.554 INFO load_module /tmp/tmpb3jgb32t/mod.so
Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
  671.1440     574.1128    1132.6036     558.6864     189.8142  
               
